# Using the geojson generator

## Import the packages

In [20]:
import pandas as pd

In [21]:
from convert2geojson import Convert2GeoJson
from SPARQLWrapper import SPARQLWrapper, JSON

In [22]:
def runQuery(query):
    sparql = SPARQLWrapper(
        "http://localhost:8080/blazegraph/namespace/kb/sparql" # This is the workshop endpoint
    )
    sparql.setReturnFormat(JSON)                     # and we want json as returns
    sparql.setQuery(query)
    try:
        ret = sparql.queryAndConvert()
        return ret["results"]["bindings"]
    except Exception as e:
        print(e)
        return 

## Load data from graph

In [25]:
linkedObjects = runQuery("""prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX crm: <http://cidoc-crm.org/cidoc-crm/>
prefix oa: <http://www.w3.org/ns/oa#>
prefix myfix: <file:///home/jupyter-mvogl/mydata/>
prefix geojson: <https://purl.org/geojson/vocab#>

SELECT  ?vals ?coord1
    FROM <http://example.org/ns/users/malte/genesis-example-geo2>
WHERE { 
      ?anno a oa:Annotation ;
  oa:hasBody / geojson:geometry / geojson:coordinates ?vals.
  ?vals rdf:first ?coord1 .
  }
"""
)


In [26]:
linkedObjects

[{'vals': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/bnode_N44b439df4e784a9882d6f794de0e0411'},
  'coord1': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '-73.32928'}},
 {'vals': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/bnode_Nd27b3948be4346368f075c865abd25ce'},
  'coord1': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '44.403333'}},
 {'vals': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/bnode_N760a37b6bc1e4b1a92585a731535b1fb'},
  'coord1': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '67.5'}},
 {'vals': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/bnode_N743842abb85b4979a29b6a62912aade6'},
  'coord1': {'datatype': 'http://www.w3.org/2001/XMLSchema#double',
   'type': 'literal',
   'value': '34.

In [27]:
linkedObjects = runQuery("""
    PREFIX crm: <http://cidoc-crm.org/cidoc-crm/>

    SELECT ?place ?river
        FROM <http://example.org/ns/users/malte/genesis-example-geo2>
    WHERE {
        ?place crm:P53_has_location ?river .
    }
    LIMIT 1000
    """
)

In [28]:
linkedObjects

[{'place': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/places/Havilah'},
  'river': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/rivers/Pishon'}},
 {'place': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/places/Cush'},
  'river': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/rivers/Gihon'}},
 {'place': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/places/Assyria'},
  'river': {'type': 'uri',
   'value': 'http://example.org/ns/users/malte/genesis-example-geo2/rivers/Hiddekel'}}]

## Build GeoJSON and save to disc

Note that not every dataframe row contains information on longitude or latitude. These rows are dropped from the dataframe. Therefore, the number of found features may vary.

Instantiate using the dataframe, a list of properties to be displayed in popups, and the column titles for the geodata. By adding `.convert()` the geojson is created directly after instantiation.

In [ ]:
build = Convert2GeoJson(dataframe=df, properties=df.columns, lat='latitude',lon='longitude').convert()

You can provide additional information for the geoJSON by adding an attribution (`attribution='Generated by ...'`) and a description (`description='Shows the ...`).


The generated geoJSON in dictionary format can be accessed by `.geojson()`.

In [ ]:
geojson = build.geojson()
geojson.keys()

To save the generated geoJSON as a file, chose

In [ ]:
#build.save('./data/testGEO.geojson')

You can also provide parameters for the filename (`name='data.geojson'`) or path (`path='.'`).

## Display result

The package uses ipyleaflet to generate the map for the geoJSON data. 

Markers on maps can be drawn fully (standard) or grouped with leaflets `MarkerCluster` (style='grouped'). The basemap for the background mapping is loaded from CartoDBs 'base-light-nolabels' source. Additonal parameters include customized basemaps or layouting for the map.

In [ ]:
mapD = build.display(style='grouped')

In [ ]:
mapD